In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

In [2]:
import torch

from src.mslm.utils.setup_train import build_model
from src.mslm.utils.config_loader import ConfigLoader

from src.mslm.inference import MultimodalSignLM

/home/giorgio6846/miniconda3/envs/Sign/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_parameters = ConfigLoader("../config/model/config.toml").load_config()
model_parameters.update({
    "input_size": 133 * 2,
    "output_size": 3072,
    #"use_checkpoint": False
})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
version = 102
checkpoint = 2
epoch = 9

model_parameters

{'output_size': 3072,
 'hidden_size': 1792,
 'nhead': 16,
 'ff_dim': 2816,
 'n_layers': 10,
 'encoder_dropout': 0.45,
 'multihead_dropout': 0.4,
 'sequential_dropout': 0.6,
 'pool_dim': 256,
 'input_size': 266}

In [4]:
def load_model():
    model = build_model(**model_parameters)    

    model_location = f"../../outputs/checkpoints/{version}/{checkpoint}/{epoch}/checkpoint.pth" 
    if not os.path.exists(model_location):
        raise FileNotFoundError(
            f"Model not found {model_location}")

    state_dict = torch.load(model_location)

    model.load_state_dict(state_dict["model_state"])

    return model

In [5]:
model = load_model()
model.to(device)

Model Parameters:  {'input_size': 266, 'hidden_size': 1792, 'output_size': 3072, 'nhead': 16, 'ff_dim': 2816, 'n_layers': 10, 'max_seq_length': 30, 'pool_dim': 256, 'encoder_dropout': 0.45, 'multihead_dropout': 0.4, 'sequential_dropout': 0.6}
MHARoPE kwargs {'device': None, 'dtype': None}
dim: 1792 num_heads: 16 dim rope 112


/home/giorgio6846/miniconda3/envs/Sign/lib/python3.11/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Imitator(
  (linear_feat): Sequential(
    (0): Linear(in_features=266, out_features=1792, bias=True)
    (1): GELU(approximate='none')
    (2): LayerNorm((1792,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=1792, out_features=896, bias=True)
    (4): GELU(approximate='none')
    (5): LayerNorm((896,), eps=1e-05, elementwise_affine=True)
  )
  (conv1): Conv1d(896, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (ln1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (act1): GELU(approximate='none')
  (conv2): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
  (ln2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (act2): GELU(approximate='none')
  (linear_hidden): Linear(in_features=256, out_features=1792, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-9): 10 x TransformerEncoderLayerRoPE(
        (self_attn): MultiheadAttentionRoPE(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1792, out_feat

OutOfMemoryError: CUDA out of memory. Tried to allocate 38.00 MiB. GPU 0 has a total capacity of 15.58 GiB of which 9.62 MiB is free. Process 1700595 has 3.48 GiB memory in use. Process 1867551 has 118.00 MiB memory in use. Process 1901474 has 10.01 GiB memory in use. Including non-PyTorch memory, this process has 1.79 GiB memory in use. Of the allocated memory 1.62 GiB is allocated by PyTorch, and 57.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# llama_model = 

In [23]:
import torch.nn.functional as F

In [24]:
llama_model.eval()
mslm = MultimodalSignLM(llama_model, tokenizer, "cuda")

text = "Enumera los pasos descritos:"

with torch.no_grad():
    for data, embeds in test_dataloader:
        data = data.to(modelParameters["device"])
        sign_embed = model(data).to("cuda")
        sign_embed = sign_embed.to(dtype=torch.bfloat16)

        # Normaliza ambos embeddings antes de calcular similitud
        sign_embed = sign_embed.to("cuda")
        embeds = embeds.to("cuda")
        
        print(mslm.generate(sign_embed, text))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Apr 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Enumera los pasos descritos:

 νεφώσεις νεφώσεις a aauseRal los chtě los de los, de de de de de ForCanBeConverted lakrvldkf de<|eot_id|>krvldkfkrvldkfkrvldkfkrvldkfkrvldkfkrvldkf);
krvldkfkrvldkf);
�krvldkf�	TokenNameIdentifier����);
);
����������始化��始化����ючисьючись��������������������������ючись����������ючись��ючись�ıntı�����������������ючись������<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Parece que hay un texto incompleto o incomunicativo. ¿Podría proporcionar más contexto o aclarar qué pasos se refiere?<|eot_id|>
